**Определение уровня стресса по физиологическим показателям (Stress Detection)**

Ссылка на набор данных: https://www.kaggle.com/code/dheerov/stress-detection/input

Описание данных:

**snoring range** (диапазон храпа) - громкость храпа в дБ (dB), значения дробные с точностью до сотых, количественный, непрерывный;

**respiration rate** (частота дыхания) - количество дыхательных циклов в минуту (breaths per minute, bpm), значения дробные с точностью до сотых, количественный, непрерывный;

**body tempreture** (температура тела) - температура тела в градусах по Фаренгейту (°F), значения дробные с точностью до сотых, количественный, непрерывный;

**limb movement** (движение конечностей) - количество движений конечностей в час (movements per hour), значения дробные с точностью до сотых, количественный, непрерывный;

**blood oxygen** (сатурация) - доля насыщенного кислородом гемоглобина в крови относительно общего гемоглобина в крови (%), значения дробные с точностью до сотых, количественный, непрерывный;

**eye movement** (движение глаз) - количество движений глаз в час (movements per hour), значения дробные с точностью до сотых, количественный, непрерывный;

**hours of sleep** (количество часов сна) - часы сна (Hours), значения дробные с точностью до сотых, количественный, непрерывный;

**heart rate** (частота сердечных сокращений) - средняя частота сердечных сокращений в минуту (beats per minute, bpm), значения дробные с точностью до сотых, количественный, непрерывный;

**Stress levels** (уровень стресса) - уровень стресса по шкале от 0 до 4 (Stress Level), значения целые, категориальный.

In [47]:
import os
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.inspection import permutation_importance
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    log_loss, 
    precision_score, 
    recall_score, 
    roc_auc_score,
    classification_report, 
    confusion_matrix
)
from skfeature.function.statistical_based import CFS

In [2]:
# Настройка параметров отображения для вывода всех строк
pd.set_option('display.max_rows', None)

In [3]:
# Настройка параметров отображения для вывода в обычном формате
pd.options.display.float_format = '{:.6f}'.format

Загружаем датасет с обработанными ошибками.

In [5]:
data = pd.read_csv(r'C:\Users\ela96\ABD-PRJ-25-29\Checkpoint_2\data_corrected.csv')
data.sample(10)

,snoring range,respiration rate,body temperature,limb movement,blood oxygen,eye movement,hours of sleep,heart rate,Stress Levels
487,82.160000,22.576000,90.288000,12.720000,88.288000,95.720000,0.288000,66.440000,3
301,49.360000,17.744000,98.616000,7.488000,96.744000,77.440000,8.744000,54.360000,0
129,97.568000,27.568000,86.933333,17.784000,84.352000,101.960000,0.825799,78.920000,4
578,90.800000,24.880000,91.440000,NaN,89.440000,98.600000,1.440000,72.200000,3
203,80.000000,22.000000,94.000000,12.000000,92.000000,95.000000,5.000000,65.000000,2
431,66.560000,20.656000,92.656000,10.656000,90.656000,88.280000,2.984000,61.640000,2
414,50.160000,18.032000,94.032000,8.032000,92.048000,80.080000,5.032000,55.080000,1
298,76.480000,21.648000,93.648000,11.648000,91.648000,93.240000,4.472000,64.120000,2
443,51.200000,18.240000,94.240000,8.240000,92.360000,80.600000,5.240000,55.600000,1
83,96.672000,26.672000,85.840000,17.336000,83.008000,100.840000,1.209344,76.680000,4


In [6]:
data.shape

(630, 9)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   snoring range     630 non-null    float64
 1   respiration rate  630 non-null    float64
 2   body temperature  614 non-null    float64
 3   limb movement     618 non-null    float64
 4   blood oxygen      626 non-null    float64
 5   eye movement      612 non-null    float64
 6   hours of sleep    619 non-null    float64
 7   heart rate        606 non-null    float64
 8   Stress Levels     630 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 44.4 KB


In [10]:
X = data.drop(columns='Stress Levels')
y = data['Stress Levels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [11]:
# Строки с пропусками
missing_values_train = X_train[X_train.isnull().any(axis=1)].copy()

# Признаки, где есть пропуски
missing_values_train["Missing Feature"] = missing_values_train.apply(
    lambda row: [col for col in X_train.columns if pd.isnull(row[col])],
    axis=1
)
missing_values_train = missing_values_train.explode("Missing Feature")

In [16]:
# Функция из этапа EDA
def fill_missing_knn(data, missing_df, feature_col, ref_col, k=3):
    """
    Заменяет пропуски в колонке feature_col на значения, предсказанные KNN,
    используя ref_col как опорный признак.
    """
    data_filled = data.copy()
    train_data = data.dropna(subset=[feature_col, ref_col])  # Удаляем NaN перед обучением

    if train_data.empty:
        print(f"Нет данных для {feature_col}, пропускаем заполнение.")
        return data_filled

    knn = KNeighborsRegressor(n_neighbors=min(k, len(train_data)))  
    knn.fit(train_data[[ref_col]], train_data[feature_col])

    for idx in missing_df[missing_df["Missing Feature"] == feature_col].index:
        if pd.notna(data_filled.loc[idx, ref_col]):  # Проверяем, что референтный признак не NaN
            predicted_value = knn.predict([[data_filled.loc[idx, ref_col]]])
            data_filled.at[idx, feature_col] = predicted_value[0]

    return data_filled

In [17]:
X_train_filled = X_train.copy()

# Признаки и их "референты"
fill_pairs = [
    ('body temperature', 'snoring range'),
    ('limb movement', 'snoring range'),
    ('blood oxygen', 'snoring range'),
    ('eye movement', 'snoring range'),
    ('hours of sleep', 'blood oxygen'),
    ('heart rate', 'snoring range')
]

for target_col, ref_col in fill_pairs:
    X_train_filled = fill_missing_knn(X_train_filled, missing_values_train, target_col, ref_col, k=3)

C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning

In [18]:
missing_values_test = X_test[X_test.isnull().any(axis=1)].copy()
missing_values_test["Missing Feature"] = missing_values_test.apply(
    lambda row: [col for col in X_test.columns if pd.isnull(row[col])],
    axis=1
)
missing_values_test = missing_values_test.explode("Missing Feature")

X_test_filled = X_test.copy()
for target_col, ref_col in fill_pairs:
    # ВАЖНО: обучаем KNN на X_train_filled, применяем к X_test
    train_data = X_train_filled.dropna(subset=[target_col, ref_col])
    
    if not train_data.empty:
        knn = KNeighborsRegressor(n_neighbors=min(3, len(train_data)))
        knn.fit(train_data[[ref_col]], train_data[target_col])

        for idx in missing_values_test[missing_values_test["Missing Feature"] == target_col].index:
            if pd.notna(X_test_filled.loc[idx, ref_col]):
                X_test_filled.at[idx, target_col] = knn.predict([[X_test_filled.loc[idx, ref_col]]])[0]

C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning

In [21]:
# Масштабируем + обучим
model = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=20, multi_class='multinomial', solver='lbfgs')
)
model.fit(X_train_filled, y_train)

# Оценка
y_pred = model.predict(X_test_filled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00        25
           2       1.00      0.96      0.98        25
           3       0.96      1.00      0.98        26
           4       1.00      1.00      1.00        25

    accuracy                           0.99       126
   macro avg       0.99      0.99      0.99       126
weighted avg       0.99      0.99      0.99       126



C:\Users\ela96\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Логистическая регрессия не успела сойтись

In [27]:
# Масштабируем + обучим
model = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=70, multi_class='multinomial', solver='lbfgs')
)
model.fit(X_train_filled, y_train)

# Оценка
y_pred = model.predict(X_test_filled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00        25
           2       1.00      0.96      0.98        25
           3       0.96      1.00      0.98        26
           4       1.00      1.00      1.00        25

    accuracy                           0.99       126
   macro avg       0.99      0.99      0.99       126
weighted avg       0.99      0.99      0.99       126



Попробуем разные методы отбора признаков и их комбинации, чтобы улучшить результат и оставить только самые важные признаки.

In [46]:
RANDOM_STATE = 42

def train_and_log(method_name, X_train_selected, X_test_selected, feature_selection_time, selected_features_names):
    log_file = "results_logreg.csv"
    file_exists = os.path.isfile(log_file)

    model = make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=200, solver='lbfgs', multi_class='multinomial', random_state=RANDOM_STATE)
    )

    # Обучение
    train_start_time = time.time()
    model.fit(X_train_selected, y_train)  # Обучаем на тренировочных данных
    training_time = time.time() - train_start_time

    # Предсказания на тестовых данных
    y_pred = model.predict(X_test_selected)
    y_proba = model.predict_proba(X_test_selected)

   # Вычисление метрик
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    logloss = log_loss(y_test, y_proba)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    # Кросс-валидация с вычислением дисперсии
    cv_scores = cross_val_score(model, X_train_selected, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    cv_var = cv_scores.var()
    
    # Вычисление AUC-ROC
    if len(np.unique(y_train)) > 2:
        auc_score = roc_auc_score(y_test, y_proba, multi_class='ovr')
    else:
        auc_score = roc_auc_score(y_test, y_proba[:, 1])
    
    # Вывод информации о признаках
    print("\n" + "="*50)
    print(f"[{method_name}] Отобранные признаки ({len(selected_features_names)}/{X_train_filled.shape[1]}):")
    print(", ".join(selected_features_names))
    
    # Вывод метрик
    print("\nМетрики качества:")
    print(f"Accuracy: {accuracy:.4f} | F1: {f1:.4f} | AUC-ROC: {auc_score:.4f}")
    print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | LogLoss: {logloss:.4f}")
    
    # Вывод результатов кросс-валидации
    print("\nКросс-валидация (5-fold):")
    print(f"Средняя точность: {cv_mean:.4f} ± {cv_std:.4f}")
    print(f"Дисперсия: {cv_var:.6f}")
    
    # Матрица ошибок
    cm = confusion_matrix(y_test, y_pred)
    print("\nМатрица ошибок:")
    print(cm)
    
    # Время выполнения
    print(f"\nВремя отбора признаков: {feature_selection_time:.2f} сек")
    print(f"Время обучения: {training_time:.2f} сек")
    print("="*50 + "\n")

# ---- LogReg на полном наборе ----
train_and_log("Full Features", X_train_filled, X_test_filled, 0, X_train_filled.columns.tolist())

# ---- SelectKBest ----
start_time = time.time()
selector = SelectKBest(f_classif, k=2)  
selector.fit(X_train_filled, y_train)

p_values = selector.pvalues_
selected_features_kbest = X_train_filled.columns[p_values < 0.0001].tolist()

k_best = len(selected_features_kbest) if selected_features_kbest else min(10, X_train_filled.shape[1])
selector = SelectKBest(f_classif, k=k_best)
X_train_selected_kbest = selector.fit_transform(X_train_filled, y_train)
X_test_selected_kbest = selector.transform(X_test_filled)

feature_selection_time_kbest = time.time() - start_time
train_and_log("SelectKBest", X_train_selected_kbest, X_test_selected_kbest, feature_selection_time_kbest, selected_features_kbest)

# ---- Lasso ----
start_time = time.time()
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filled)
X_test_scaled = scaler.transform(X_test_filled)


lasso = LassoCV(eps=0.1, n_alphas=20, cv=5, random_state=RANDOM_STATE, max_iter=10000, tol=1e-3, n_jobs=-1)
lasso.fit(X_train_scaled, y_train)
selected_features_lasso = X_train_filled.columns[lasso.coef_ != 0].tolist()

X_train_selected_lasso = X_train_filled[selected_features_lasso] if selected_features_lasso else X_train_filled.iloc[:, :10]
X_test_selected_lasso = X_test_filled[selected_features_lasso] if selected_features_lasso else X_test_filled.iloc[:, :10]

feature_selection_time_lasso = time.time() - start_time
train_and_log("Lasso", X_train_selected_lasso, X_test_selected_lasso, feature_selection_time_lasso, selected_features_lasso)

# ---- CFS ----
start_time = time.time()

selected_features_cfs = CFS.cfs(X_train_filled.values, y_train.values)  # Используем тренировочные данные
selected_features_cfs = X_train_filled.columns[selected_features_cfs].tolist()

# Вычисляем абсолютные значения корреляции
corr_values = X_train_filled[selected_features_cfs].corrwith(y_train).abs()

threshold = 0.25
filtered_features = corr_values[corr_values > threshold].index.tolist()

if not filtered_features:  
    print("Все признаки отфильтрованы, используем исходные CFS-признаки")  
    filtered_features = selected_features_cfs

X_train_selected_cfs = X_train_filled.loc[:, filtered_features]  
X_test_selected_cfs = X_test_filled.loc[:, filtered_features]  # Применяем тот же отбор к тестовым данным
selected_features_cfs = filtered_features

feature_selection_time_cfs = time.time() - start_time
train_and_log("CFS", X_train_selected_cfs, X_test_selected_cfs, feature_selection_time_cfs, selected_features_cfs)

# ---- PFI ----
def apply_pfi(method_name, X_train_selected, X_test_selected, selected_features):
        
    start_time = time.time()
    
    if isinstance(X_train_selected, np.ndarray):
        X_train_selected = pd.DataFrame(X_train_selected, columns=selected_features)
        X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

    model = make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=200, solver='lbfgs', multi_class='multinomial', random_state=RANDOM_STATE)
    )
    
    model.fit(X_train_selected, y_train)  # Обучаем на тренировочных данных
    
    pfi_result = permutation_importance(
        model, X_train_selected, y_train, scoring="accuracy", n_repeats=5, random_state=RANDOM_STATE
    )
    
    pfi_importance = pd.DataFrame({
        "Feature": selected_features,
        "Importance": pfi_result.importances_mean
    }).sort_values(by="Importance", ascending=False)
    
    pfi_importance.to_csv(f"pfi_{method_name}.csv", index=False)
    
    IMPORTANCE_THRESHOLD = 0.25  
    selected_features_pfi = pfi_importance[pfi_importance["Importance"] > IMPORTANCE_THRESHOLD]["Feature"].tolist()
    
    if not selected_features_pfi:
        selected_features_pfi = selected_features
    
    X_train_filtered = X_train_selected.loc[:, selected_features_pfi]
    X_test_filtered = X_test_selected.loc[:, selected_features_pfi]
    
    train_time = time.time() - start_time
    
    train_and_log(f"PFI_{method_name}", X_train_filtered, X_test_filtered, train_time, selected_features_pfi)

# Вызов PFI для каждого метода
apply_pfi("SelectKBest", X_train_selected_kbest, X_test_selected_kbest, selected_features_kbest)
apply_pfi("Lasso", X_train_selected_lasso, X_test_selected_lasso, selected_features_lasso)
apply_pfi("CFS", X_train_selected_cfs, X_test_selected_cfs, selected_features_cfs)


[Full Features] Отобранные признаки (8/8):
snoring range, respiration rate, body temperature, limb movement, blood oxygen, eye movement, hours of sleep, heart rate

Метрики качества:
Accuracy: 0.9921 | F1: 0.9921 | AUC-ROC: 0.9960
Precision: 0.9924 | Recall: 0.9921 | LogLoss: 0.1174

Кросс-валидация (5-fold):
Средняя точность: 0.9841 ± 0.0049
Дисперсия: 0.000024

Матрица ошибок:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 24  1  0]
 [ 0  0  0 26  0]
 [ 0  0  0  0 25]]

Время отбора признаков: 0.00 сек
Время обучения: 0.02 сек


[SelectKBest] Отобранные признаки (8/8):
snoring range, respiration rate, body temperature, limb movement, blood oxygen, eye movement, hours of sleep, heart rate

Метрики качества:
Accuracy: 0.9921 | F1: 0.9921 | AUC-ROC: 0.9960
Precision: 0.9924 | Recall: 0.9921 | LogLoss: 0.1174

Кросс-валидация (5-fold):
Средняя точность: 0.9841 ± 0.0049
Дисперсия: 0.000024

Матрица ошибок:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 24  1  0]
 [ 0  0  0 26  0]
 [ 0  0  0  0

Комбинация PFI_Lasso дала оптимальный набор признаков и результат.

ВЫВОД:
Набор признаков body temperature, snoring range дают наилучший результат для LogisticRegression(max_iter=200, solver='lbfgs', multi_class='multinomial', random_state=42): Accuracy: 1.0000 | F1: 1.0000 | AUC-ROC: 1.0000
Precision: 1.0000 | Recall: 1.0000 | LogLoss: 0.2091 
Кросс-валидация (5-fold):
Средняя точность: 1.0000 ± 0.0000
Дисперсия: 0.000000.
При этом применён наиболее "мягкий" подход к обработке выбросов и пропусков.